## Data Imports

In [1]:
import pandas as pd
import numpy as np
import requests
import ujson
import os

import plotly
from plotly import tools

import plotly
with open('/Users/timlee/Dropbox/keys/plotly_apikey.txt','r') as f:
    api_key = f.read()
plotly.tools.set_credentials_file(username='tdlee', api_key=api_key)

import plotly.plotly as ply
import plotly.graph_objs as go
from plotly.grid_objs import Grid, Column

### Data Preprocessing

In [3]:
country_codes = ['chn', 'idn', 'ind', 'jpn', 'sau', 'tur', 'twn',
                 'bel', 'che', 'deu', 'esp', 'fra', 'gbr', 'ita',
                 'nld', 'pol', 'rus', 'swe', 'can', 'mex', 'usa',
                 'aus', 'arg', 'bra', 'tha']


def get_countries_lkup():
    countries_url = 'https://atlas.media.mit.edu/attr/country/'
    resp = requests.get(countries_url)
    resp_json = ujson.loads(resp.content)['data']
    resp_df = pd.DataFrame(resp_json)
    country_id_lkup = {idx: name for idx, name in resp_df[['display_id','name']].values}
    return country_id_lkup


def get_data(countrycode):
    imports_url = './OEC_%s_imports.feather' % countrycode
    exports_url = './OEC_%s_exports.feather' % countrycode
    
    if not os.path.isfile(imports_url):
        return None, None
    
    if not os.path.isfile(exports_url):
        return None, None
        
    
    df_exports = pd.read_feather(exports_url)
    df_exports = df_exports[df_exports['export_val'].isna()==False].copy()
    df_exports = df_exports.sort_values(by='export_val')
    
    df_imports = pd.read_feather(imports_url)
    df_imports = df_imports[df_imports['import_val'].isna() == False].copy()
    df_imports = df_imports.sort_values(by = 'import_val')
    
    return df_imports, df_exports



def all_ports(country_codes):
    import_collector = []
    export_collector = []
    for countrycode in country_codes:
        df_imports, df_exports = get_data(countrycode)
        if df_imports is not None:
            df_imp_tmp = df_imports[['year','product_name','import_val']].copy()
            df_imp_tmp['countrycode'] = countrycode
            df_exp_tmp = df_exports[['year','product_name','export_val']].copy()
            df_exp_tmp['countrycode'] = countrycode
            import_collector.append(df_imp_tmp)
            export_collector.append(df_exp_tmp)
        
    return pd.concat(import_collector), pd.concat(export_collector)
    

### Creating Import Ratio Chart

In [9]:
df_summary_imp = import_collector.groupby(['countrycode','year'])['import_val'].sum().reset_index()
df_summary_exp = export_collector.groupby(['countrycode','year'])['export_val'].sum().reset_index()
df_total= df_summary_imp.merge(df_summary_exp, on=['year','countrycode'])
df_total['imp_ratio'] = df_total['export_val'] / df_total['import_val']
df_total.sort_values('imp_ratio', inplace=True, ascending=False)

traces = []
titles = []
for countrycode in df_total['countrycode'].unique():
    titles.append(country_id_lkup[countrycode])
    mask = df_total['countrycode'] == countrycode
    tmp = df_total[mask].copy()
    tmp = tmp.sort_values('year')
    tr = go.Scatter(
        x = tmp['year'].values,
        y = tmp['imp_ratio'].values,
        mode = 'lines',
        line = dict(width=5),
        name = country_id_lkup[countrycode]
    )
    traces.append(tr)
    
font_dict = dict(
            family='Roboto',
            size=20
        )

layout = go.Layout(
    font = font_dict,
    hovermode='closest',
    width=900,
    height=600,
    xaxis=dict(
        title='Year',
        titlefont=font_dict,
        tickfont=font_dict,        
    ),
    yaxis=dict(
        title='Import Ratio',
        type = 'log',
        titlefont=font_dict,
        tickfont=font_dict,
    ),
)

fig = dict(data=traces, layout=layout)
ply.iplot(fig, filename='import_ratio')

### Get top Country codes

In [36]:
countries_lkup = get_countries_lkup()
mask = df_total['year'] == 2010
tmp = df_total[mask].sort_values(by='imp_ratio', ascending=False)
top20_codes = list(tmp['countrycode'].values)
top20_codes = top20_codes[:20]
titles = [countries_lkup[cd] for cd in top20_codes]

In [37]:
top20_codes

['sau',
 'rus',
 'chn',
 'idn',
 'arg',
 'aus',
 'jpn',
 'deu',
 'bra',
 'mex',
 'swe',
 'che',
 'can',
 'nld',
 'ita',
 'fra',
 'pol',
 'esp',
 'tur',
 'gbr']

### Create small multiples chart for Export / Import ratio

In [39]:
subplots = tools.make_subplots(rows=5, cols=4, subplot_titles=titles)

traces = []
titles = []
for countrycode in top20_codes:
    titles.append(country_id_lkup[countrycode])
    mask = df_total['countrycode'] == countrycode
    tmp = df_total[mask].copy()
    tmp = tmp.sort_values('year')
    tr = go.Scatter(
        x = tmp['year'].values,
        y = tmp['imp_ratio'].values,
        mode = 'lines',
        line = dict(width=5),
        name = country_id_lkup[countrycode]
    )
    traces.append(tr)
    
counter = 0
for i in range(5):
    for j in range(4):
        subplots.append_trace(traces[counter], i+1, j+1)
        counter += 1
        subplots['layout']['yaxis' + str(counter)].update(type='log')
        subplots['layout']['yaxis' + str(counter)].update(range=[-0.4,.7])
        
subplots['layout'].update(title="Top Countries Export / Import Ratio")
subplots['layout'].update(font=dict(family='Roboto', size=12))

for d in subplots['layout']['annotations']:
    d['font']['size']=12

# yaxis=dict(
#     title='Import Ratio',
#     type = 'log',
# )

subplots['layout'].update(height=1000, width=1000)
subplots['layout'].update(showlegend=False)
?subplots['layout'].update(yaxis=yaxis)

ply.iplot(subplots, filename='Top_Countries_imp_exp_ratio_multiples')

This is the format of your plot grid:
[ (1,1) x1,y1 ]    [ (1,2) x2,y2 ]    [ (1,3) x3,y3 ]    [ (1,4) x4,y4 ]  
[ (2,1) x5,y5 ]    [ (2,2) x6,y6 ]    [ (2,3) x7,y7 ]    [ (2,4) x8,y8 ]  
[ (3,1) x9,y9 ]    [ (3,2) x10,y10 ]  [ (3,3) x11,y11 ]  [ (3,4) x12,y12 ]
[ (4,1) x13,y13 ]  [ (4,2) x14,y14 ]  [ (4,3) x15,y15 ]  [ (4,4) x16,y16 ]
[ (5,1) x17,y17 ]  [ (5,2) x18,y18 ]  [ (5,3) x19,y19 ]  [ (5,4) x20,y20 ]



### Create Imports Heatmap

In [32]:
df_summary_imp = import_collector.groupby(['product_name','year'])['import_val'].sum().reset_index().sort_values('import_val')
df_summary_exp = export_collector.groupby(['product_name','year'])['export_val'].sum().reset_index().sort_values('export_val')

mask = (df_summary_imp['year'] == 2014) & (df_summary_imp['import_val'] > 0)
top_100_imp_prod = df_summary_imp.loc[mask,'product_name'].tail(50)

mask = (df_summary_exp['year'] == 2014) & (df_summary_exp['export_val'] > 0)
top_100_exp_prod = df_summary_exp.loc[mask,'product_name'].tail(50)

mask = import_collector['product_name'].isin(top_100_imp_prod)
df_summary_imp = import_collector[mask].groupby(['product_name','countrycode','year'])['import_val'].sum().reset_index().sort_values('import_val')

mask = export_collector['product_name'].isin(top_100_exp_prod)
df_summary_exp = export_collector[mask].groupby(['product_name','countrycode','year'])['export_val'].sum().reset_index().sort_values('export_val')

In [40]:
year = 2014
mask = df_summary_imp['year'] == year
piv_data = df_summary_imp[mask].pivot(index='product_name', columns='countrycode', values='import_val' )
piv_data = piv_data.fillna(0)
tr = go.Heatmap(
    z = piv_data.as_matrix(),
    x = piv_data.columns,
    y = piv_data.index,
    colorscale = 'Viridis'
)

In [41]:
layout = go.Layout(
    width=900,
    height=900,
    title='2014 Top Imports by Country',
    margin=dict(
        l=200
    )
)
fig = dict(data=[tr], layout=layout)
ply.iplot(fig, filename='2014_top_import_heatmap')

### Create Exports Heatmap

In [42]:
year = 2014
mask = df_summary_exp['year'] == year
piv_data = df_summary_exp[mask].pivot(index='product_name', columns='countrycode', values='export_val' )
piv_data = piv_data.fillna(0)
tr = go.Heatmap(
    z = piv_data.as_matrix(),
    x = piv_data.columns,
    y = piv_data.index,
    colorscale = 'Viridis'
)

In [43]:
layout = go.Layout(
    width=900,
    height=900,
    title='2014 Top Exports by Country',
    margin=dict(
        l=200
    )
)
fig = dict(data=[tr], layout=layout)
ply.iplot(fig, filename='2014_top_export_heatmap')